# Домашнее задание 3

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена.
Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам “Война и мир” по-русски и по-английски:

In [2]:
path_to_AnnaKarenina = "data/AnnaKarenina.txt"
path_to_WarAndPeace = "data/WarAndPeace.txt"
path_to_WarAndPeaceEng = "data/WarAndPeaceEng.txt"

## Часть 1

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
    * подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
    * возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
    * расшифруйте их таким частотным методом.


In [215]:
import os
import re
import random
import string
import numpy as np
from copy import copy
from collections import Counter, defaultdict
from tqdm import tqdm
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


In [129]:
#support function

def read_data(path):
    with open(path, "r", encoding="utf8") as file:
        data = file.read().replace('\n', ' ')
        data = re.sub('\W+',' ', data).lower()
    return data

def show_first_elem(d, i):
    cnt = 0
    for k, v in d:
        cnt+=1
        print(f'\"{k}\" :  {v}')
        if cnt == i:
            break

In [16]:
AnnaKarenina = read_data(path_to_AnnaKarenina)
WarAndPeace = read_data(path_to_WarAndPeace)
WarAndPeaceEng = read_data(path_to_WarAndPeaceEng)

In [14]:
AnnaKarenina[:150]

'Annotation   «Анна Каренина», один из самых знаменитых романов Льва Толстого, начинается ставшей афоризмом фразой: «Все счастливые семьи похожи друг н'

Возьмем из Анны Карениной тестовые тексты это будут 500, 1000 и 2000 первых символов и в качестве бонуса на всем сете.

In [96]:
AnnaKarenina[:1000]

'annotation анна каренина один из самых знаменитых романов льва толстого начинается ставшей афоризмом фразой все счастливые семьи похожи друг на друга каждая несчастливая семья несчастлива по своему это книга о вечных ценностях о любви о вере о семье о человеческом достоинстве лев толстойроман широкого дыхания часть перваяi лев толстой анна каренина роман широкого дыхания анна каренина поразила современников вседневностью содержания необычайная свобода раскованность повествования удивительно сочетались в этом романе с цельностью художественного взгляда автора на жизнь он выступал здесь как художник и мыслитель и назначение искусства видел не в том чтобы неоспоримо разрешить вопрос а в том чтобы заставить любить жизнь в бесчисленных никогда не истощимых всех ее проявлениях 61 100 1 в 70 е годы один маститый писатель по видимому гончаров сказал достоевскому это вещь неслыханная это вещь первая кто у нас из писателей может поравняться с этим а в европе кто представит хоть что нибудь подоб

In [89]:
def text_encoder(text):
    
    chars_in_text = list(set(text))
    
    cipher = list(set(string.printable))
    random.shuffle(cipher)
    cipher = cipher[:len(chars_in_text)]
    mapping = dict(zip(chars_in_text, cipher)) 
    ciphertext = [mapping[char] for char in text]
    
    return "".join(ciphertext)


In [165]:
text1 = AnnaKarenina[:500]
text2 = AnnaKarenina[:1000]
text3 = AnnaKarenina[:2000]

ciphertext1 = text_encoder(text1)
ciphertext2 = text_encoder(text2)
ciphertext3 = text_encoder(text3)
ciphertext4 = text_encoder(AnnaKarenina)

In [166]:
ciphertext1

"zTT>$z$U>T4y\x0c\x0cy4ey%@\x0cX\x0cy4<\tX\x0c4X{4\ry'=\\4{\x0cy'@\x0cX~=\\4%<'y\x0c<?4Im?y4~<I\r~< <4\x0cyvX\x0cy@~\r,4\r~y?s@K4yL<%X{'<'4L%y{<K4?\r@4\rvy\r~IX?=@4\r@'mX4!<\\<QX4\t%F 4\x0cy4\t%F y4eyQ\ty,4\x0c@\rvy\r~IX?y,4\r@'m,4\x0c@\rvy\r~IX?y4!<4\r?<@'F4D~<4e\x0cX y4<4?@v\x0c=\\4;@\x0c\x0c<\r~,\\4<4IR*?X4<4?@%@4<4\r@'m@4<4v@I<?@v@\re<'4\t<\r~<X\x0c\r~?@4I@?4~<I\r~<K%<'y\x0c4sX%<e< <4\t=\\y\x0cX,4vy\r~m4!@%?y,U4I@?4~<I\r~<K4y\x0c\x0cy4ey%@\x0cX\x0cy4%<'y\x0c4sX%<e< <4\t=\\y\x0cX,4y\x0c\x0cy4ey%@\x0cX\x0cy4!<%y{XIy4\r<?%@'@\x0c\x0cXe<?4?\r@\t\x0c@?\x0c<\r~mR4\r<\t@%Qy\x0cX,4\x0c@<*=vyK\x0cy,4\r?<*<\ty4%y\re<?y\x0c\x0c<\r~m4!<?@\r~?<?y\x0cX,4F\tX?X~@Im\x0c<4\r<v@"

Зашифровали, теперь надо бы получить статистику по корпусу

In [167]:
def text_letter_stats(text):
    counts = dict(Counter(text))
    counts = sorted(counts.items(), key=lambda item: item[1], reverse=True )
    counts = [(k, v/len(text)) for k, v in counts]

    return counts


In [168]:
AnnaKarenina_stats = text_letter_stats(AnnaKarenina)
show_first_elem(AnnaKarenina_stats, 10)

" " :  0.16759116836318097
"о" :  0.09513034595302555
"е" :  0.07242743491488532
"а" :  0.06859314466860275
"н" :  0.05748448067215471
"и" :  0.05498627598220739
"т" :  0.04957691600291936
"с" :  0.044003547274936064
"л" :  0.04153755858919674
"в" :  0.038988394038047686


Теперь приступим к расшифровке:

In [169]:
def text_decoder(text, global_stats):
    
    text_count = Counter(text)
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True)
    
    map_count = {}
    for i,value in enumerate(text_count):
        map_count[value[0]] = global_stats[i][0]
    
    
    decoded_text = []
    
    for item in text:
        decoded_text.append(map_count[item])
    
    return "".join(decoded_text)
        
    

In [178]:
jojo = text_decoder(ciphertext3, AnnaKarenina_stats)

In [177]:
jojo[:1000]


'2ee8ъ2ъn8e аееа дарнеиеа омие иб такьж беакнеисьж рокаеол впла совтсояо еагиеанстч тсалхнз аiорибкок iрабоз лтн тгатсвильн тнкпи ыожоши мруя еа мруяа дашмач ентгатсвилач тнкпч ентгатсвила ыо тлонку юсо деияа о лнгеьж щнееотсчж о вэйли о лнрн о тнкпн о гнволнгнтдок мотсоиетслн внл совтсозрокае хиродояо мьжаеич гатсп ынрлачn внл совтсоз аееа дарнеиеа рокае хиродояо мьжаеич аееа дарнеиеа ыорабива толрнкнееидол лтнменлеотспэ томнршаеич енойьгазеач тлойома ратдолаееотсп ыолнтслолаеич умилиснвпео тогнсавитп л юсок рокаен т щнвпеотспэ жумошнтслнееояо лбявчма алсора еа шибеп ое льтсуыав бмнтп дад жумошеид и кьтвиснвп и еабеагнеин итдуттсла лимнв ен л сок гсойь енотыорико рабрнхисп лоырот а л сок гсойь батсалисп вэйисп шибеп л йнтгитвнееьж еидояма ен итсофикьж лтнж нн ырочлвнеичж 1ц цaa ц л sa н яомь омие катсисьз ыитаснвп ыо лимикоку яоегарол тдабав мотсонлтдоку юсо лнфп ентвьжаееач юсо лнфп ынрлач дсо у еат иб ыитаснвнз кошнс ыоралечсптч т юсик а л нлроын дсо ырнмтсалис жосп гсо еийумп ыомой

Собственно прочитать что то осмысленое сложно. попробуем посчитать метрики цифрами 

In [180]:
def accuracy_metric(true, pred):
    return (np.array(list(true)) == np.array(list(pred))).mean()

In [188]:
metric1 = accuracy_metric(text1, text_decoder(ciphertext1, AnnaKarenina_stats))
metric2 = accuracy_metric(text2, text_decoder(ciphertext2, AnnaKarenina_stats))
metric3 = accuracy_metric(text3, text_decoder(ciphertext3, AnnaKarenina_stats))

print(f'Метрика accuracy на выборке размером в 500: {metric1}')
print(f'Метрика accuracy на выборке размером в 1000: {metric2}')
print(f'Метрика accuracy на выборке размером в 2000: {metric3}')


Метрика accuracy на выборке размером в 500: 0.27
Метрика accuracy на выборке размером в 1000: 0.262
Метрика accuracy на выборке размером в 2000: 0.425


## Часть 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.

In [191]:
def bigram_maker(text):
    bigram_text = []
    for i in  range(len(text) - 1):
        bigram_text.append(text[i:i+2])
    return bigram_text

In [192]:
def text_bigram_stats(text):
    counts = dict(Counter(bigram_maker(text)))
    counts = sorted(counts.items(), key=lambda item: item[1], reverse=True )
    counts = [(k, v/len(text)) for k, v in counts]

    return counts

In [197]:
AnnaKarenina_bigram_stats = text_bigram_stats(AnnaKarenina)
len(AnnaKarenina_bigram_stats)

1346

In [199]:
AnnaKarenina_bigram_stats[:10]

[('о ', 0.023729137208547666),
 ('е ', 0.018688211168292893),
 ('а ', 0.01853181711150135),
 ('и ', 0.017949000308102148),
 (' н', 0.016254438486761565),
 (' с', 0.0159967104530976),
 (' в', 0.014667653843135004),
 ('то', 0.014148683302620743),
 (' п', 0.013940157893565352),
 (' о', 0.013363198545476697)]

In [222]:
def bigram_text_decoder(text, global_bigram_stats):
    n_gram = 2
    text_count = Counter(bigram_maker(text))
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True)
    
    
    map_count = {}
    
    for i, (text_ngram, text_freq) in enumerate(text_count):
        global_bigram_stats_copy  = copy(global_bigram_stats)
        

        for j in range(n_gram):
            if text_ngram[j] in map_count:
                global_bigram_stats_copy = [
                    (ngram, freq)
                    for ngram, freq in global_bigram_stats_copy
                    if ngram[j] == map_count[text_ngram[j]]
                ]

        min_diff = 1.0  # maximum possible frequency
        best_ngram = None
        for ngram, freq in global_bigram_stats_copy:
            diff = abs(freq - text_freq)
            if diff < min_diff:
                best_ngram = ngram
                min_diff = diff

        for j in range(n_gram):
            if text_ngram[j] not in map_count:
                map_count[text_ngram[j]] = best_ngram[j]

    return map_count
    
    
    

In [223]:
text_count = Counter(text)
    text_count = sorted(text_count.items(), key=lambda item: item[1], reverse=True)

IndentationError: unexpected indent (3158594432.py, line 2)

In [224]:
reverse_mapping_bigram = bigram_text_decoder(ciphertext1, AnnaKarenina_bigram_stats)

TypeError: 'NoneType' object is not subscriptable